# Examen ETL: SPARK 02/02

In [252]:
sc.stop()

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de partidosLigaNBA.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera pregunta: Describe brevemente que diferencia el persists, cache y collect en spark. Explica brevemente casos en los que es interesante su aplicación

## Segunda pregunta: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

## Tercera Pregunta: Índica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

## Cuarta tarea: Inicializar spark context y cargar los datos desde el fichero.

In [253]:
from pyspark import SparkContext
sc = SparkContext()

In [254]:
NBA = sc.textFile("partidosLigaNBA.csv",)
NBA.take(10)

['Date:Start..ET.:Visitor.Neutral:PTS:Home.Neutral:PTS.1',
 'Tue, Oct 30, 2007:"7:30 pm":Utah Jazz:117:Golden State Warriors:96',
 'Tue, Oct 30, 2007:"7:30 pm":Houston Rockets:95:Los Angeles Lakers:93',
 'Tue, Oct 30, 2007:"7:00 pm":Portland Trail Blazers:97:San Antonio Spurs:106',
 'Wed, Oct 31, 2007:"8:00 pm":Dallas Mavericks:92:Cleveland Cavaliers:74',
 'Wed, Oct 31, 2007:"8:30 pm":Seattle SuperSonics:103:Denver Nuggets:120',
 'Wed, Oct 31, 2007:"7:00 pm":Washington Wizards:110:Indiana Pacers:119',
 'Wed, Oct 31, 2007:"7:00 pm":San Antonio Spurs:104:Memphis Grizzlies:101',
 'Wed, Oct 31, 2007:"7:30 pm":Chicago Bulls:103:New Jersey Nets:112',
 'Wed, Oct 31, 2007:"7:00 pm":Sacramento Kings:90:New Orleans Hornets:104']

In [279]:
NBA_filteres = NBA.filter(lambda x: x!= 'Date:Start..ET.:Visitor.Neutral:PTS:Home.Neutral:PTS.1')
NBA_filteres2 = NBA_filteres.filter(lambda x: x!= 'Playoffs:Playoffs:Playoffs:Playoffs:Playoffs:Playoffs')

In [294]:
NBA_parse_tempo = NBA_filteres2.map(lambda x: (x[0:27], x.split(':')))

In [295]:
NBA_parse_tempo.take(2)

[('Tue, Oct 30, 2007:"7:30 pm"',
  ['Tue, Oct 30, 2007',
   '"7',
   '30 pm"',
   'Utah Jazz',
   '117',
   'Golden State Warriors',
   '96']),
 ('Tue, Oct 30, 2007:"7:30 pm"',
  ['Tue, Oct 30, 2007',
   '"7',
   '30 pm"',
   'Houston Rockets',
   '95',
   'Los Angeles Lakers',
   '93'])]

In [296]:
NBA_parse1 = NBA_parse_tempo.map(lambda x: (x[0], x[1][3], x[1][4],x[1][5],x[1][6]))
NBA_parse = NBA_parse1.map(lambda x: (x[0].split(','),x[1],x[2],x[3],x[4]))

In [297]:
NBA_parse.take(10)

[(['Tue', ' Oct 30', ' 2007:"7:30 pm"'],
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96'),
 (['Tue', ' Oct 30', ' 2007:"7:30 pm"'],
  'Houston Rockets',
  '95',
  'Los Angeles Lakers',
  '93'),
 (['Tue', ' Oct 30', ' 2007:"7:00 pm"'],
  'Portland Trail Blazers',
  '97',
  'San Antonio Spurs',
  '106'),
 (['Wed', ' Oct 31', ' 2007:"8:00 pm"'],
  'Dallas Mavericks',
  '92',
  'Cleveland Cavaliers',
  '74'),
 (['Wed', ' Oct 31', ' 2007:"8:30 pm"'],
  'Seattle SuperSonics',
  '103',
  'Denver Nuggets',
  '120'),
 (['Wed', ' Oct 31', ' 2007:"7:00 pm"'],
  'Washington Wizards',
  '110',
  'Indiana Pacers',
  '119'),
 (['Wed', ' Oct 31', ' 2007:"7:00 pm"'],
  'San Antonio Spurs',
  '104',
  'Memphis Grizzlies',
  '101'),
 (['Wed', ' Oct 31', ' 2007:"7:30 pm"'],
  'Chicago Bulls',
  '103',
  'New Jersey Nets',
  '112'),
 (['Wed', ' Oct 31', ' 2007:"7:00 pm"'],
  'Sacramento Kings',
  '90',
  'New Orleans Hornets',
  '104'),
 (['Wed', ' Oct 31', ' 2007:"7:00 pm"'],
  'Milwaukee Bucks'

## Quinta tarea: Media de la diferencia de puntos por año

In [299]:
NBA_year_diff = NBA_parse.map(lambda x: (x[0][2][1:5], (int(abs(float(x[2])-float(x[4]))), 1)))
NBA_year_diff.take(10)

[('2007', (21, 1)),
 ('2007', (2, 1)),
 ('2007', (9, 1)),
 ('2007', (18, 1)),
 ('2007', (17, 1)),
 ('2007', (9, 1)),
 ('2007', (3, 1)),
 ('2007', (9, 1)),
 ('2007', (14, 1)),
 ('2007', (19, 1))]

In [300]:
NBA_year_diff_2 = NBA_year_diff.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).map(lambda x: (x[0], x[1][0]/x[1][1]))
NBA_year_diff_2.take(15)

[('2017', 11.422166874221668),
 ('2013', 11.071752265861027),
 ('2016', 11.550637659414853),
 ('2011', 10.661016949152541),
 ('2014', 10.9047976011994),
 ('2008', 11.543543543543544),
 ('2007', 11.096491228070175),
 ('2015', 11.159969673995452),
 ('2009', 11.090425531914894),
 ('2010', 10.86903860711582),
 ('2012', 10.845318860244234)]

## Sexta tarea: ¿Han judado todos los equipos el mismo número de partidos? ¿ Si es qué no a que puede deberse?

In [319]:
NBA_Team = NBA_parse.map(lambda x: (x[1],1)).reduceByKey(lambda x,y: x+y).join(
    NBA_parse.map(lambda x: (x[3],1)).reduceByKey(lambda x,y: x+y))

In [326]:
NBA_Team.map(lambda x: (x[0], x[1][0]+x[1][0])).collect()

[('New Orleans Pelicans', 332),
 ('Charlotte Hornets', 254),
 ('Phoenix Suns', 826),
 ('New Jersey Nets', 394),
 ('Golden State Warriors', 880),
 ('Brooklyn Nets', 436),
 ('Milwaukee Bucks', 828),
 ('San Antonio Spurs', 932),
 ('Toronto Raptors', 848),
 ('New Orleans Hornets', 498),
 ('Utah Jazz', 848),
 ('Los Angeles Lakers', 894),
 ('Indiana Pacers', 868),
 ('Houston Rockets', 868),
 ('Dallas Mavericks', 872),
 ('Oklahoma City Thunder', 816),
 ('Washington Wizards', 846),
 ('Cleveland Cavaliers', 904),
 ('Memphis Grizzlies', 868),
 ('Chicago Bulls', 874),
 ('Denver Nuggets', 848),
 ('Portland Trail Blazers', 856),
 ('Miami Heat', 912),
 ('Seattle SuperSonics', 82),
 ('Sacramento Kings', 804),
 ('Philadelphia 76ers', 836),
 ('New York Knicks', 826),
 ('Orlando Magic', 864),
 ('Boston Celtics', 926),
 ('Atlanta Hawks', 896),
 ('Detroit Pistons', 828),
 ('Minnesota Timberwolves', 804),
 ('Charlotte Bobcats', 566),
 ('Los Angeles Clippers', 860)]

## Séptima pregunta: ¿Cuantos partidos ha ganado en Enero Clevelant?

In [332]:
NBA_Cleveland_visit = NBA_parse.filter(lambda x: 'Jan' in x[0][1]).filter(lambda x: x[1] == 'Cleveland Cavaliers')
NBA_Cleveland_home = NBA_parse.filter(lambda x: 'Jan' in x[0][1]).filter(lambda x: x[3] == 'Cleveland Cavaliers')

In [341]:
NBA_Cleveland_visit_win = NBA_Cleveland_visit.filter(lambda x: int(x[2])>int(x[4]))
NBA_Cleveland_home_win = NBA_Cleveland_home.filter(lambda x: int(x[4])>int(x[2]))

In [344]:
NBA_Cleveland_home_win.take(1)

[(['Wed', ' Jan 2', ' 2008:"7:00 pm":'],
  'Atlanta Hawks',
  '94',
  'Cleveland Cavaliers',
  '98')]

In [343]:
NBA_Cleveland_home_win.count()+NBA_Cleveland_visit_win.count()

83

## Octava pregunta: ¿Los Warrios son mejores fuera de casa o en casa?

In [355]:
print("Los Warrios han ganado",
      NBA_parse.filter(lambda x: x[1] == 'Golden State Warriors').filter(lambda x: int(x[2])>int(x[4])).count(),
      "de visitantes")

Los Warrios han ganado 215 de visitantes


In [356]:
print("Por otro lado, Warrios han ganado", 
      NBA_parse.filter(lambda x: x[3] == 'Golden State Warriors').filter(lambda x: int(x[4])>int(x[2])).count(), 
      "en casa, por lo cual son mejores en casa")

Por otro lado, Warrios han ganado 308 en casa, por lo cual son mejores en casa


## Novena pregunta: Equipo que ha quedado primerio en victorias más temporadas. (si es que hay alguno que más)

In [357]:
NBA_parse.take(1)

[(['Tue', ' Oct 30', ' 2007:"7:30 pm"'],
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96')]

In [370]:
# Primero filtramos ganadores como vitisantes por año
NBA_year_winner_visit = NBA_parse.filter(lambda x: int(x[2])>int(x[4])).map(lambda x: ((x[0][2][1:5], x[1]), 1))
NBA_year_winner_visit.take(10)

[(('2007', 'Utah Jazz'), 1),
 (('2007', 'Houston Rockets'), 1),
 (('2007', 'Dallas Mavericks'), 1),
 (('2007', 'San Antonio Spurs'), 1),
 (('2007', 'Detroit Pistons'), 1),
 (('2007', 'Phoenix Suns'), 1),
 (('2007', 'Houston Rockets'), 1),
 (('2007', 'Philadelphia 76ers'), 1),
 (('2007', 'Denver Nuggets'), 1),
 (('2007', 'Toronto Raptors'), 1)]

In [372]:
# Ahora filtramos por ganadores en casa
NBA_year_winner_home = NBA_parse.filter(lambda x: int(x[4])>int(x[2])).map(lambda x: ((x[0][2][1:5], x[3]),1))
NBA_year_winner_home.take(10)

[(('2007', 'San Antonio Spurs'), 1),
 (('2007', 'Denver Nuggets'), 1),
 (('2007', 'Indiana Pacers'), 1),
 (('2007', 'New Jersey Nets'), 1),
 (('2007', 'New Orleans Hornets'), 1),
 (('2007', 'Orlando Magic'), 1),
 (('2007', 'Toronto Raptors'), 1),
 (('2007', 'Atlanta Hawks'), 1),
 (('2007', 'Boston Celtics'), 1),
 (('2007', 'Charlotte Bobcats'), 1)]

In [374]:
NBA_year_winner_visit_sum = NBA_year_winner_visit.reduceByKey(lambda x,y: x+y)
NBA_year_winner_home_sum = NBA_year_winner_home.reduceByKey(lambda x,y: x+y)

In [378]:
NBA_year_winner = NBA_year_winner_home_sum.join(NBA_year_winner_visit_sum)
NBA_year_winner.take(10)

[(('2013', 'Los Angeles Clippers'), (32, 22)),
 (('2013', 'Indiana Pacers'), (43, 24)),
 (('2012', 'Minnesota Timberwolves'), (22, 18)),
 (('2007', 'Atlanta Hawks'), (10, 5)),
 (('2007', 'Sacramento Kings'), (8, 3)),
 (('2017', 'Dallas Mavericks'), (15, 8)),
 (('2016', 'Brooklyn Nets'), (15, 5)),
 (('2007', 'Dallas Mavericks'), (14, 6)),
 (('2010', 'San Antonio Spurs'), (36, 27)),
 (('2011', 'New Orleans Hornets'), (17, 14))]

In [385]:
NBA_year_winner_count = NBA_year_winner.map(lambda x: (x[0][0],x[0][1],(x[1][0]+x[1][1])))
NBA_year_winner_count.take(10)

[('2013', 'Los Angeles Clippers', 54),
 ('2013', 'Indiana Pacers', 67),
 ('2012', 'Minnesota Timberwolves', 40),
 ('2007', 'Atlanta Hawks', 15),
 ('2007', 'Sacramento Kings', 11),
 ('2017', 'Dallas Mavericks', 23),
 ('2016', 'Brooklyn Nets', 20),
 ('2007', 'Dallas Mavericks', 20),
 ('2010', 'San Antonio Spurs', 63),
 ('2011', 'New Orleans Hornets', 31)]

In [388]:
NBA_year_count = NBA_year_winner_count.map(lambda x: (x[0],x[2]))
NBA_year_count.take(10)

[('2013', 54),
 ('2013', 67),
 ('2012', 40),
 ('2007', 15),
 ('2007', 11),
 ('2017', 23),
 ('2016', 20),
 ('2007', 20),
 ('2010', 63),
 ('2011', 31)]

In [405]:
NBA_year_count_max = NBA_year_count.reduceByKey(max).map(lambda x: ((x[0],x[1]),''))
NBA_year_count_max.collect()

[(('2017', 54), ''),
 (('2013', 85), ''),
 (('2011', 53), ''),
 (('2016', 87), ''),
 (('2014', 73), ''),
 (('2008', 84), ''),
 (('2007', 26), ''),
 (('2015', 88), ''),
 (('2009', 81), ''),
 (('2010', 71), ''),
 (('2012', 82), '')]

In [414]:
NBA_year_count_key = NBA_year_winner_count.map(lambda x: ((x[0], x[2]), x[1]))
NBA_year_count_key.take(10)

[(('2013', 54), 'Los Angeles Clippers'),
 (('2013', 67), 'Indiana Pacers'),
 (('2012', 40), 'Minnesota Timberwolves'),
 (('2007', 15), 'Atlanta Hawks'),
 (('2007', 11), 'Sacramento Kings'),
 (('2017', 23), 'Dallas Mavericks'),
 (('2016', 20), 'Brooklyn Nets'),
 (('2007', 20), 'Dallas Mavericks'),
 (('2010', 63), 'San Antonio Spurs'),
 (('2011', 31), 'New Orleans Hornets')]

In [412]:
NBA_year_count_max.join(NBA_year_count_key).collect()

[(('2009', 81), ('', 'Los Angeles Lakers')),
 (('2007', 26), ('', 'Boston Celtics')),
 (('2015', 88), ('', 'Golden State Warriors')),
 (('2012', 82), ('', 'San Antonio Spurs')),
 (('2017', 54), ('', 'Golden State Warriors')),
 (('2010', 71), ('', 'Los Angeles Lakers')),
 (('2008', 84), ('', 'Boston Celtics')),
 (('2014', 73), ('', 'San Antonio Spurs')),
 (('2016', 87), ('', 'Golden State Warriors')),
 (('2013', 85), ('', 'Miami Heat')),
 (('2011', 53), ('', 'Chicago Bulls'))]

##  Décima pregunta: Escribe la expresión regular correcta que sólo macheen los teléfonos y el correo del siguiente texto.

Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com

In [ ]:
"Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com"

In [ ]:
(\w\w\w\s\w\w\s\w\w\s\w\w)